In [1]:
import numpy as np
import pandas as pd

from time import process_time
from tqdm.auto import tqdm

In [24]:
import requests
from bs4 import BeautifulSoup 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [38]:
path = 'C:/Users/s0970/python/701網路爬蟲/chromedriver.exe'

In [66]:
driver = webdriver.Chrome(path) #開啟

<ipython-input-66-60bd7adf2b89>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟


## 試抓某個縣市某個行政區的資料

In [194]:
driver.get("https://referendum.2021.nat.gov.tw/pc/zh_TW/IDX/indexFF.html")

driver.find_element_by_name("nodeIcon2").click()
driver.find_element_by_name("nodeIcon3").click()
driver.find_element_by_id("itemTextLink4").click()

html = BeautifulSoup(driver.page_source, 'html.parser')



<ipython-input-194-ec32d294270f>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon2").click()
<ipython-input-194-ec32d294270f>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon3").click()
<ipython-input-194-ec32d294270f>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("itemTextLink4").click()


In [195]:
import re

#### 縣市、行政區

In [196]:
html.select('td[valign="bottom"]')[0].text

'第17案\xa0全國性\xa0公民投票結果\xa0-\xa0臺北市松山區'

In [197]:
pattern = re.compile(r'(?<=\xa0-\xa0)\w+')
print(pattern.findall(html.select('td[valign="bottom"]')[0].text)[0])

Adm = pattern.findall(html.select('td[valign="bottom"]')[0].text)[0]

臺北市松山區


#### 同意票數、不同意票數、有效票數、無效票數、投票數、投票權人數、投票率(%) 有效同意票數對投票權人數(%)

In [121]:
html.select('tr.trT td')

[<td>42,583</td>,
 <td>37,246</td>,
 <td>79,829</td>,
 <td>743</td>,
 <td>80,572</td>,
 <td>162,964</td>,
 <td>49.44%</td>,
 <td>26.13%</td>]

#### 已完成投票所

In [122]:
html.select('td[align="right"]')[0].text

'投開票所數 已送/應送:\xa0134/134\xa0'

In [118]:
pattern = re.compile(r'(?<=\xa0)\d+')
print(pattern.findall(html.select('td[align="right"]')[0].text)[0])

vote = int(pattern.findall(html.select('td[align="right"]')[0].text)[0])

134


In [125]:
df = [i.text for i in html.select('tr.trT td')]

df.insert(0,Adm)
df.append(vote)

df

['臺北市松山區',
 '42,583',
 '37,246',
 '79,829',
 '743',
 '80,572',
 '162,964',
 '49.44%',
 '26.13%',
 134]

## 找出各縣市行政區的關聯性

In [133]:
driver.get("https://referendum.2021.nat.gov.tw/pc/zh_TW/IDX/indexFF.html")

driver.find_element_by_name("nodeIcon2").click()

<ipython-input-133-c88071ff9ad5>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon2").click()


In [134]:
html = BeautifulSoup(driver.page_source, 'html.parser')

In [151]:
html.select('div[id="domRoot"] div[id]')

[<div id="folder0" style="display:block;"><table border="0" cellpadding="0" cellspacing="0" width="100%"><tbody><tr><td valign="top"><img height="2" src="../images/ftv2blank.gif" width="2"/></td><td valign="middle" width="100%"><a>查詢項目</a></td></tr></tbody></table></div>,
 <div id="item1" style="display: block;"><table border="0" cellpadding="0" cellspacing="0" width="100%"><tbody><tr><td background="../images/ftv2vertline.gif" valign="top"><img height="22" src="../images/ftv2node.gif" width="16"/></td><td valign="top"><img height="2" src="../images/ftv2blank.gif" width="3"/></td><td valign="middle" width="100%"><a href="../00/00000000000000000.html" id="itemTextLink1" onclick="clickOnLink('1', '../00/00000000000000000.html','_self');return false;" target="_self">全國性公投案結果</a></td></tr></tbody></table></div>,
 <div id="folder2" style="display: block;"><table border="0" cellpadding="0" cellspacing="0" width="100%"><tbody><tr><td background="../images/ftv2vertline.gif" valign="top"><a hre

In [149]:
number = []
for i in html.select('div[id="domRoot"] div[id]'):
    number.append(i.attrs["id"])
    

### 取出各縣市網頁起始值

In [163]:
number[3:25]

['folder3',
 'folder16',
 'folder46',
 'folder60',
 'folder90',
 'folder128',
 'folder167',
 'folder181',
 'folder200',
 'folder227',
 'folder241',
 'folder262',
 'folder281',
 'folder315',
 'folder328',
 'folder342',
 'folder359',
 'folder366',
 'folder373',
 'folder378',
 'folder386',
 'folder390']

In [173]:
pattern = re.compile(r'(?<=folder)\d+')

num = [int(pattern.findall(i)[0]) for i in number[3:25]]
print("縣市數量:{}".format(len(num)))
num 


縣市數量:22


[3,
 16,
 46,
 60,
 90,
 128,
 167,
 181,
 200,
 227,
 241,
 262,
 281,
 315,
 328,
 342,
 359,
 366,
 373,
 378,
 386,
 390]

In [186]:
numbers=[]

for j in range(len(num)-1): # 最後一個嘉義市另外看
    numbers.append([num[j]+i+1 for i in range(num[j+1]-num[j]-1)]) 

In [187]:
numbers.append([391,392]) # 嘉義

In [188]:
np.array(numbers)

array([list([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]),
       list([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]),
       list([47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]),
       list([61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]),
       list([91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]),
       list([129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166]),
       list([168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180]),
       list([182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]),
       list([201, 202, 203, 204, 

# 開始抓取資料

In [229]:
driver = webdriver.Chrome(path) 

<ipython-input-229-ba3f382862d7>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [230]:
def vote_data(city,adms):

    driver.get("https://referendum.2021.nat.gov.tw/pc/zh_TW/IDX/indexFF.html")

    driver.find_element_by_name("nodeIcon2").click()
    driver.find_element_by_name("nodeIcon{}".format(city)).click()
    driver.find_element_by_id("itemTextLink{}".format(adms)).click()

    html = BeautifulSoup(driver.page_source, 'html.parser')


    pattern = re.compile(r'(?<=\xa0-\xa0)\w+')
    Adm = pattern.findall(html.select('td[valign="bottom"]')[0].text)[0]

    pattern = re.compile(r'(?<=\xa0)\d+')
    vote = int(pattern.findall(html.select('td[align="right"]')[0].text)[0])

    df = [i.text for i in html.select('tr.trT td')]

    df.insert(0,Adm)
    df.append(vote)

    return df


In [258]:
k=-1
for i in num:
    k+=1
    for j in numbers[k]:
        print([i,j])

[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[3, 10]
[3, 11]
[3, 12]
[3, 13]
[3, 14]
[3, 15]
[16, 17]
[16, 18]
[16, 19]
[16, 20]
[16, 21]
[16, 22]
[16, 23]
[16, 24]
[16, 25]
[16, 26]
[16, 27]
[16, 28]
[16, 29]
[16, 30]
[16, 31]
[16, 32]
[16, 33]
[16, 34]
[16, 35]
[16, 36]
[16, 37]
[16, 38]
[16, 39]
[16, 40]
[16, 41]
[16, 42]
[16, 43]
[16, 44]
[16, 45]
[46, 47]
[46, 48]
[46, 49]
[46, 50]
[46, 51]
[46, 52]
[46, 53]
[46, 54]
[46, 55]
[46, 56]
[46, 57]
[46, 58]
[46, 59]
[60, 61]
[60, 62]
[60, 63]
[60, 64]
[60, 65]
[60, 66]
[60, 67]
[60, 68]
[60, 69]
[60, 70]
[60, 71]
[60, 72]
[60, 73]
[60, 74]
[60, 75]
[60, 76]
[60, 77]
[60, 78]
[60, 79]
[60, 80]
[60, 81]
[60, 82]
[60, 83]
[60, 84]
[60, 85]
[60, 86]
[60, 87]
[60, 88]
[60, 89]
[90, 91]
[90, 92]
[90, 93]
[90, 94]
[90, 95]
[90, 96]
[90, 97]
[90, 98]
[90, 99]
[90, 100]
[90, 101]
[90, 102]
[90, 103]
[90, 104]
[90, 105]
[90, 106]
[90, 107]
[90, 108]
[90, 109]
[90, 110]
[90, 111]
[90, 112]
[90, 113]
[90, 114]
[90, 115]
[90, 116]
[90, 117]
[90, 118]


In [259]:
Final_data=[]

k=-1
for i in num:
    k+=1
    for j in numbers[k]:
        Final_data.append(vote_data(i,j))

<ipython-input-230-ddc14a75dec9>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon2").click()
<ipython-input-230-ddc14a75dec9>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon{}".format(city)).click()
<ipython-input-230-ddc14a75dec9>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("itemTextLink{}".format(adms)).click()


In [264]:
final_data = pd.DataFrame(Final_data)
final_data.columns=["縣市行政區","同意票數","不同意票數","有效票數","無效票數","投票數","投票權人數","已完成投票所投票率(%)","有效同意票數對投票權人數(%)","已完成投票所"]

In [265]:
final_data

,縣市行政區,同意票數,不同意票數,有效票數,無效票數,投票數,投票權人數,已完成投票所投票率(%),有效同意票數對投票權人數(%),已完成投票所
0,臺北市松山區,"42,583","37,246","79,829",743,"80,572","162,964",49.44%,26.13%,134
1,臺北市信義區,"45,522","38,628","84,150",843,"84,993","181,747",46.76%,25.05%,166
2,臺北市大安區,"64,621","52,374","116,995","1,132","118,127","242,869",48.64%,26.61%,194
3,臺北市中山區,"41,082","45,181","86,263",794,"87,057","188,193",46.26%,21.83%,144
4,臺北市中正區,"31,561","27,558","59,119",584,"59,703","123,767",48.24%,25.5%,115
...,...,...,...,...,...,...,...,...,...,...
363,新竹市東區,"42,096","34,243","76,339",891,"77,230","169,450",45.58%,24.84%,160
364,新竹市北區,"27,138","25,882","53,020",657,"53,677","125,506",42.77%,21.62%,122
365,新竹市香山區,"12,213","13,399","25,612",353,"25,965","66,397",39.11%,18.39%,64
366,嘉義市東區,"19,300","23,106","42,406",311,"42,717","100,157",42.65%,19.27%,89


In [270]:
final_data.to_csv("vote_data.csv",encoding='utf-8-sig')